In [1]:
import docx2txt
import xlrd
import pandas as pd
import re 
import numpy as np
import os
from tqdm import tqdm
import sys
import copy

In [2]:
option_sheet_name = 'Sheet3'
option_sheet_index = 2

# option_sheet_name = 'Sheet2'
# option_sheet_index = 1


In [3]:
tmp_file_path = r"./tmp.txt"

In [4]:
excel = pd.read_excel(r"./data/decoration/装修手册_故障代码.xls",header=None,sheet_name=option_sheet_name)
excel

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,S-1 起动性能差,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,起动性能差的一般原因,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,•,电气系统故障,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,•,燃油供应不足,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,•,进气不足,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,•,燃油选择不合适,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,★ 共用油槽喷油系统（CRI）控制电子喷油正时。因此， 即使进行了起动操作，直到曲轴转动 2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
workbook = xlrd.open_workbook(r"./data/decoration/装修手册_故障代码.xls",formatting_info=True)

sheet = workbook.sheet_by_index(option_sheet_index)

In [6]:
synonymMap = {'报警级别':['报警级别'], '故障代码':['故障代码'], '故障':['故障'], '故障内容':['故障内容'],  \
              '控制器的反应':['控制器的反应'], '机器发生的故障':['机器发生的故障'],  \
             '相关信息':['相关信息'], '起因':['起因'], '故障诊断':['故障诊断']}

name_to_en = {"故障代码":"faultCode", "故障名称":"faultName", "报警级别":"alertLevel","故障内容":"faultContent", 
                 "控制器的反应":"controllerReaction", "机器发生的故障":"machineFault", "相关信息":"relatedInfo", 
                 "起因":"cause", "故障诊断":"diagnosisAction", "标准值":"standardVal", "附图":"image", 
                  "检查系统":"monitorSystem", "故障":"faultName", "步骤数":"stepNum", "问题和检查项目":"problem_and_check_item",
                 "措施":"solution"}
name_to_en = {k: v.lower() for k, v in name_to_en.items()}
name_to_ch = {v: k for k, v in name_to_en.items()}
name_to_ch

{'faultcode': '故障代码',
 'faultname': '故障',
 'alertlevel': '报警级别',
 'faultcontent': '故障内容',
 'controllerreaction': '控制器的反应',
 'machinefault': '机器发生的故障',
 'relatedinfo': '相关信息',
 'cause': '起因',
 'diagnosisaction': '故障诊断',
 'standardval': '标准值',
 'image': '附图',
 'monitorsystem': '检查系统',
 'stepnum': '步骤数',
 'problem_and_check_item': '问题和检查项目',
 'solution': '措施'}

In [7]:
excel.loc[14,0]

'问题'

## 切分出每个故障

In [8]:
# 适用于一般的故障代码

code_line_num = []
for i in range(len(excel)):
#     print(i)
#     re.match(r'故障代码 \[\w*\].*',str(excel[i][0])) is not None:
    match_result = re.match(r'([S]-[\d|_]+)\s*(.*)',str(excel.loc[i,0])) 
    if match_result is not None:
#         print(excel.loc[i,0])
#         print(match_result.group(1), match_result.group(2))
        code_line_num.append(i)
code_line_num.append(len(excel))
print(len(code_line_num))
code_line_num
        

18


[0,
 48,
 83,
 120,
 164,
 209,
 256,
 288,
 343,
 394,
 434,
 470,
 535,
 573,
 608,
 652,
 701,
 735]

In [9]:
len(code_line_num)

18

In [10]:
for line_num in code_line_num[:-1]:
    print(excel.loc[line_num,0])

S-1 起动性能差
S-2_1 发动机不能起动:发动机不转
S-2_2 发动机不能起动:发动机转动但是不排烟
S-2_3 发动机不能起动:排烟冒出但发动机不起动（燃油被喷射）
S-3 发动机加速不平稳
S-4 发动机操作期间停机
S-5 发动机运行剧烈不稳定
S-6 发动机动力不足
S-7 排烟为黑色（燃烧不充分）
S-8 发动机油耗过多（或排烟为蓝色）
S-9 发动机油污染得快
S-10 燃油消耗过多
S-12 发动机油压下降
S-13 机油油位升高（冷却液或燃油进入）
S-14 冷却液温度变得太高（过热）
S-15 发出异常噪音
S-16 振动过度


## 逐个读取出每个实体/属性的值

In [11]:
excel.shape[1]

20

In [12]:
# 适用于一般的故障代码
fc_dicts = {}
for i_code in range(len(code_line_num)-1):
#     if i_code == 3:
#         break
    fc_dict = {}
    i_start = code_line_num[i_code]
    i_end = code_line_num[i_code+1]
#     print(str(excel.loc[i,0]))
    match_result = re.match(r'([S]-[\d|_]+)\s*(.*)',str(excel.loc[i_start,0])) 
    fc_code = match_result.group(1)
    fc_content = match_result.group(2)
    fc_dict['故障代码'] = fc_code
    fc_dict['故障'] = fc_content
    print("i range in (%d,%d)"%(i_start, i_end))
#     print(excel.loc[i_start:i_end])

    for i in range(i_start, i_end):
        first_col_content = excel.loc[i,0]
        if str(first_col_content).endswith('一般原因') or str(first_col_content).endswith('原因'):
            fc_dict['一般起因'] = []
            i_row = i + 1
            value_now = excel.loc[i_row,0]
            while value_now != value_now:
                i_row += 1
                value_now = excel.loc[i_row,0]
            while value_now == value_now:
                if value_now != '.' and value_now != '·' and value_now != '•':
                    value_use = value_now
                else:
                    value_use = excel.loc[i_row,1]
                fc_dict['一般起因'].append(value_use)
                i_row += 1
                value_now = excel.loc[i_row,0]
        if first_col_content == '问题':
            cause = []
            for j in range(2,excel.shape[1]):
                value_now = excel.loc[i-1,j]
                if value_now == value_now:
                    cause.append(value_now)
            fc_dict['起因'] = cause
            i_content_start = i 
            i_content_end = i
            for i_content_end in range(i,i+25,1):
                cell = sheet.cell(i_content_end,0)
                fmt = workbook.xf_list[cell.xf_index]
                file = open(tmp_file_path, mode='w')
                fmt.dump(file)
                file.close()
                file = open(tmp_file_path,mode='r')
                dump_dic = {}
                for line in file.readlines():
                    match_result = re.match('(.*):\s(\d)', line)
                    if match_result is not None:
                        key = match_result.group(1).strip()
                        value = match_result.group(2)
                        dump_dic[key] = int(value)
                file.close()
                if dump_dic['bottom_line_style'] == 2:
                    break
#             print(i_content_start,i_content_end, excel.loc[i_content_start,0], excel.loc[i_content_end,0], excel.loc[i_content_end,3])
            problem_check_item = []
            for i_content in range(i_content_start, i_content_end+1):
                a_problem = excel.loc[i_content,2]
                if a_problem != a_problem:
                    a_problem = last_problem
                last_problem = a_problem
                second_problem_column = excel.loc[i_content,3]
                if second_problem_column == second_problem_column:
                    a_problem += second_problem_column
                problem_check_item.append(a_problem)
            fc_dict['问题和检查项目'] = problem_check_item
            problem_cause_matrix = np.zeros((len(problem_check_item),len(cause)),dtype=int)
            for i_row in range(len(problem_check_item)):
                for i_col in range(len(cause)):
                    value_now = excel.loc[i_row+i, i_col+4]
#                     print(value_now)
#                     print(i_row, i_col, excel.loc[i_row,i_col])
                    if value_now == '△':
                        problem_cause_matrix[i_row,i_col] = 1
                    elif value_now == '○':
                        problem_cause_matrix[i_row,i_col] = 2
                    elif value_now == '◎':
                        problem_cause_matrix[i_row,i_col] = 3
                    
                        
#             print(problem_cause_matrix)
            fc_dict['problem_cause_matrix'] = problem_cause_matrix
            
        if first_col_content == '故障诊断':
            i_content_start = i 
            i_content_end = i
            for i_content_end in range(i,i+25,1):
                cell = sheet.cell(i_content_end,0)
                fmt = workbook.xf_list[cell.xf_index]
                file = open(tmp_file_path, mode='w')
                fmt.dump(file)
                file.close()
                file = open(tmp_file_path,mode='r')
                dump_dic = {}
                for line in file.readlines():
                    match_result = re.match('(.*):\s(\d)', line)
                    if match_result is not None:
                        key = match_result.group(1).strip()
                        value = match_result.group(2)
                        dump_dic[key] = int(value)
                file.close()
                if dump_dic['bottom_line_style'] == 2:
                    break
#             print(i_content_start,i_content_end, excel.loc[i_content_start,0], excel.loc[i_content_end+1,0], excel.loc[i_content_end+1,3])
            
            if excel.loc[i_content_end+1,2] == '措施':
                solution_col_start = 3
            elif excel.loc[i_content_end+1,3] == '措施':
                solution_col_start = 4
            else:
                raise Exception
            diagnosisAction = []
            for i_content in range(i_content_start, i_content_end+1):
                a_diagnosis = excel.loc[i_content,1]
                if a_diagnosis != a_diagnosis:
                    a_diagnosis = last_diagnosis
                last_diagnosis = a_diagnosis
                second_diagnosis_column = excel.loc[i_content,2]
                if second_diagnosis_column == second_diagnosis_column:
                    a_diagnosis += second_diagnosis_column
                diagnosisAction.append(a_diagnosis)
#             print(solution_col_start)
            fc_dict['故障诊断'] = diagnosisAction
            solution = []
            for j_content in range(solution_col_start, solution_col_start + len(fc_dict['起因'])):
                aSolution = excel.loc[i_content_end+1, j_content]
                solution.append(aSolution)
            fc_dict['措施'] = solution
            diagnosis_cause_matrix = np.zeros((len(diagnosisAction),len(cause)),dtype=int)
            for i_row in range(len(diagnosisAction)):
                for i_col in range(len(cause)):
                    value_now = excel.loc[i_row+i, i_col+solution_col_start]
#                     print(value_now)
#                     print(i_row, i_col, excel.loc[i_row,i_col])
                    if value_now == '●':
                        diagnosis_cause_matrix[i_row,i_col] = 1
#             print(diagnosis_cause_matrix)
            fc_dict['diagnosis_cause_matrix'] = diagnosis_cause_matrix
#     print(fc_dict)
    fc_dicts[fc_dict['故障代码']] = fc_dict
fc_dicts
    
    

i range in (0,48)
i range in (48,83)
i range in (83,120)
i range in (120,164)
i range in (164,209)
i range in (209,256)
i range in (256,288)
i range in (288,343)
i range in (343,394)
i range in (394,434)
i range in (434,470)
i range in (470,535)
i range in (535,573)
i range in (573,608)
i range in (608,652)
i range in (652,701)
i range in (701,735)


{'S-1': {'故障代码': 'S-1',
  '故障': '起动性能差',
  '一般起因': ['电气系统故障',
   '燃油供应不足',
   '进气不足',
   '燃油选择不合适',
   '★ 共用油槽喷油系统（CRI）控制电子喷油正时。因此， 即使进行了起动操作，直到曲轴转动 2 圈时，发动机可 能才起动。然而，此现象并不表示有故障。'],
  '起因': ['空气滤清器滤芯堵塞',
   '气门和气门座接触不良',
   '活塞环、气缸磨损',
   '燃油箱盖的通气孔堵塞',
   '燃油管路泄漏或堵塞、空气进入',
   '燃油滤清器、滤芯堵塞',
   '供油泵柱塞粘连、卡滞',
   '喷油器故障',
   '进气加热器系统故障',
   '交流发电机（调节器部分）故障',
   '交流发电机（励磁部分）故障',
   '蓄电池故障或老化'],
  '问题和检查项目': ['确认近期的修理历史',
   '机器工作小时数已工作了很长一段时间',
   '起动性能突然变糟',
   '起动性能当温暖时，发动机容易起动',
   '使用了超出规定的燃油',
   '还没有根据操作保养手册进行滤清器的更换',
   '必须更频繁地加注机油',
   '当发动机预热时或温度低时，预热监控器显示不正常',
   '操作期间，充电量监控器显示充电异常',
   '灰尘指示灯为红色',
   '燃油箱盖的通气孔堵塞',
   '燃油管路漏油',
   '当操作注油泵时，它没有反应或很重',
   '起动马达缓慢地转动发动机',
   '当用起动马达转 动发动机时如果拆下了燃油滤清器的排气螺塞，燃油不流出',
   '当用起动马达转 动发动机时如果断开了喷油器的溢流软管，溢出的燃油很少',
   '一起动发动机，就触摸排气歧管时，某些气缸的的温度低',
   '发动机加速不平稳且燃烧不规律',
   '发动机振动',
   '窜气过多'],
  'problem_cause_matrix': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
         [2, 3, 3, 0, 0, 2, 0, 0, 0, 0, 

In [13]:
np.zeros((3,4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

## 将故障值从字典形式转化为关系表形式

In [14]:
fc_dicts.keys()

dict_keys(['S-1', 'S-2_1', 'S-2_2', 'S-2_3', 'S-3', 'S-4', 'S-5', 'S-6', 'S-7', 'S-8', 'S-9', 'S-10', 'S-12', 'S-13', 'S-14', 'S-15', 'S-16'])

In [15]:
fc_dicts['S-1'].keys()

dict_keys(['故障代码', '故障', '一般起因', '起因', '问题和检查项目', 'problem_cause_matrix', '故障诊断', '措施', 'diagnosis_cause_matrix'])

In [16]:
problem_cause_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 2],
       [0, 2, 0, 2, 0, 2, 0, 0],
       [0, 2, 0, 2, 0, 0, 0, 0],
       [0, 3, 0, 3, 0, 0, 0, 0],
       [0, 3, 0, 3, 0, 0, 0, 0],
       [0, 2, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 2],
       [0, 0, 2, 0, 0, 2, 2, 2],
       [3, 0, 0, 0, 2, 0, 0, 0]])

In [17]:
a, b = problem_cause_matrix.shape
print(a)
print(b)

11
8


In [18]:
merge = pd.DataFrame(columns=name_to_en.keys())
for fc_dict_oral in fc_dicts.values():
#     print(fc_dict)
    fc_dict = copy.deepcopy(fc_dict_oral)
    list_keys = []
    for key in fc_dict.keys():
#         print(type(fc_dict[key]))
        if isinstance(fc_dict[key], list):
            list_keys.append(key)
        if isinstance(fc_dict[key],np.ndarray):
            list_keys.append(key)
#     print(list_keys)
    for key in list_keys:
        fc_dict.pop(key, None)
    print(fc_dict)
    single_fc_df = pd.DataFrame(fc_dict, index=[0])
    
    key = '起因'
    key_prop = '措施'
    fc_dict = {}
    fc_dict[key] = fc_dict_oral[key]
    fc_dict[key_prop] = fc_dict_oral[key_prop]
    fc_dict['故障代码'] = [fc_dict_oral['故障代码']] * len(fc_dict_oral[key])
    fc_dict['possibility'] = ['特殊']  * len(fc_dict_oral[key])
    tmp_df = pd.DataFrame(fc_dict)
    single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    key = '问题和检查项目'
    fc_dict = {}
    fc_dict[key] = fc_dict_oral[key]
    fc_dict['故障代码'] = [fc_dict_oral['故障代码']] * len(fc_dict_oral[key])
    tmp_df = pd.DataFrame(fc_dict)
    single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    key_row = '问题和检查项目'
    key_col = '起因'
    possibility_key = 'possibility_problem_cause'
    fc_dict = copy.deepcopy(fc_dict_oral)
#     print(fc_dict)
    problem_cause_matrix = fc_dict['problem_cause_matrix']
    len_1, len_2 = problem_cause_matrix.shape
    for i in range(len_1):
        for j in range(len_2):
            if problem_cause_matrix[i][j] > 0:
                fc_dict_problem_cause = {}
                fc_dict_problem_cause[key_row] = fc_dict[key_row][i]
                fc_dict_problem_cause[key_col] = fc_dict[key_col][j]
                fc_dict_problem_cause[possibility_key] = problem_cause_matrix[i][j]
                tmp_df = pd.DataFrame(fc_dict_problem_cause,  index=[0])
                single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    key_row = '故障诊断'
    key_col = '起因'
    possibility_key = 'possibility_problem_cause'
    fc_dict = copy.deepcopy(fc_dict_oral)
    diagnosis_cause_matrixa = fc_dict['diagnosis_cause_matrix']
    len_1, len_2 = diagnosis_cause_matrixa.shape
    for i in range(len_1):
        for j in range(len_2):
            if diagnosis_cause_matrixa[i][j] > 0:
                fc_dict_diagnosis_cause = {}
                fc_dict_diagnosis_cause[key_row] = fc_dict[key_row][i]
                fc_dict_diagnosis_cause[key_col] = fc_dict[key_col][j]
                fc_dict_diagnosis_cause[possibility_key] = diagnosis_cause_matrixa[i][j]
                tmp_df = pd.DataFrame(fc_dict_diagnosis_cause,  index=[0])
                single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
                
                
    key = '一般起因'
    key_changed = '起因'
    fc_dict = {}
#     print(fc_dict_oral)
    fc_dict[key_changed] = fc_dict_oral[key]
    fc_dict['故障代码'] = [fc_dict_oral['故障代码']] * len(fc_dict_oral[key])
    fc_dict['possibility'] = ['一般']  * len(fc_dict_oral[key])
    tmp_df = pd.DataFrame(fc_dict)
    single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
#     fc_dict = copy.deepcopy(fc_dict_oral)
#     # fc_dict_list = {}
#     for key in list_keys:
#         if key == '诊断操作' or key == '标准值' or key == '步骤数':
#             continue
#         fc_dict_list = {}
# #         print(fc_dict)
#         fc_dict_list['故障代码'] = [fc_dict['故障代码']] * len(fc_dict[key])
#         fc_dict_list[key] = fc_dict[key]
# #         print(fc_dict_list)
# #         print(pd.DataFrame(fc_dict_list))
#         tmp_df = pd.DataFrame(fc_dict_list)
#     #     a = pd.concat((a,b))
#         single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
#     fc_dict = copy.deepcopy(fc_dict_oral)
#     cause_dict = {}
#     cause_dict['起因'] = []
#     cause_dict['诊断操作'] = []
#     cause_dict['步骤数'] = []
# #     print(fc_dict)
#     for i in range(len(fc_dict['起因'])):
#         cause = fc_dict['起因'][i]
#         diagnosisAction = fc_dict['诊断操作'][i]
#         stepNum = fc_dict['步骤数'][i]
# #         print("debug:cause=",cause)
# #         print("debug:diagnosisAction=",diagnosisAction)
#         cause_dict['起因'] += [cause] * len(diagnosisAction)
#         cause_dict['诊断操作'] += diagnosisAction
#         cause_dict['步骤数'] += stepNum
# #         cause_dict['起因'].append(cause)
# #         cause_dict['诊断操作'].append(diagnosisAction)
# #     print(cause_dict)
#     if len(cause_dict) == 1:
#         tmp_df = pd.DataFrame(cause_dict, index=[0])
#     else:
#         tmp_df = pd.DataFrame(cause_dict)
    
#     single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    merge = pd.concat((merge,single_fc_df), sort=True)
#     print("*"*80)
merge

{'故障代码': 'S-1', '故障': '起动性能差'}
{'故障代码': 'S-2_1', '故障': '发动机不能起动:发动机不转'}
{'故障代码': 'S-2_2', '故障': '发动机不能起动:发动机转动但是不排烟'}
{'故障代码': 'S-2_3', '故障': '发动机不能起动:排烟冒出但发动机不起动（燃油被喷射）'}
{'故障代码': 'S-3', '故障': '发动机加速不平稳'}
{'故障代码': 'S-4', '故障': '发动机操作期间停机'}
{'故障代码': 'S-5', '故障': '发动机运行剧烈不稳定'}
{'故障代码': 'S-6', '故障': '发动机动力不足'}
{'故障代码': 'S-7', '故障': '排烟为黑色（燃烧不充分）'}
{'故障代码': 'S-8', '故障': '发动机油耗过多（或排烟为蓝色）'}
{'故障代码': 'S-9', '故障': '发动机油污染得快'}
{'故障代码': 'S-10', '故障': '燃油消耗过多'}
{'故障代码': 'S-12', '故障': '发动机油压下降'}
{'故障代码': 'S-13', '故障': '机油油位升高（冷却液或燃油进入）'}
{'故障代码': 'S-14', '故障': '冷却液温度变得太高（过热）'}
{'故障代码': 'S-15', '故障': '发出异常噪音'}
{'故障代码': 'S-16', '故障': '振动过度'}


,possibility,possibility_problem_cause,报警级别,控制器的反应,措施,故障,故障代码,故障内容,故障名称,故障诊断,机器发生的故障,标准值,检查系统,步骤数,相关信息,起因,问题和检查项目,附图
0,NaN,NaN,NaN,NaN,NaN,起动性能差,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,特殊,NaN,NaN,NaN,清洁,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,空气滤清器滤芯堵塞,NaN,NaN
1,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,气门和气门座接触不良,NaN,NaN
2,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,活塞环、气缸磨损,NaN,NaN
3,特殊,NaN,NaN,NaN,清洁,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油箱盖的通气孔堵塞,NaN,NaN
4,特殊,NaN,NaN,NaN,修正,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油管路泄漏或堵塞、空气进入,NaN,NaN
5,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油滤清器、滤芯堵塞,NaN,NaN
6,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,供油泵柱塞粘连、卡滞,NaN,NaN
7,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,喷油器故障,NaN,NaN
8,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,进气加热器系统故障,NaN,NaN


In [19]:
merge = merge.rename(columns=name_to_en)
merge.to_csv("./data/decoration/manual_s.csv",quoting=1,sep=';',index=False,encoding='utf-16')
merge

,possibility,possibility_problem_cause,alertlevel,controllerreaction,solution,faultname,faultcode,faultcontent,faultname,diagnosisaction,machinefault,standardval,monitorsystem,stepnum,relatedinfo,cause,problem_and_check_item,image
0,NaN,NaN,NaN,NaN,NaN,起动性能差,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,特殊,NaN,NaN,NaN,清洁,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,空气滤清器滤芯堵塞,NaN,NaN
1,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,气门和气门座接触不良,NaN,NaN
2,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,活塞环、气缸磨损,NaN,NaN
3,特殊,NaN,NaN,NaN,清洁,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油箱盖的通气孔堵塞,NaN,NaN
4,特殊,NaN,NaN,NaN,修正,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油管路泄漏或堵塞、空气进入,NaN,NaN
5,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,燃油滤清器、滤芯堵塞,NaN,NaN
6,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,供油泵柱塞粘连、卡滞,NaN,NaN
7,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,喷油器故障,NaN,NaN
8,特殊,NaN,NaN,NaN,更换,NaN,S-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,进气加热器系统故障,NaN,NaN
